In [88]:
import pandas as pd

In [89]:
failure_data = pd.read_csv("../project_data/drainage_failures.csv",names=["Description","Priority"])

In [90]:
failure_data

,Description,Priority
0,1 x 600 x 600 sunken and collapsing gully requ...,28DY
1,***This needs to be a E600*** Gully grating da...,28DY
2,(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage...,28DY
3,(Gully) 1 No new lid required for seized gully...,28DY
4,(Gully) 1 No. new lid required. Corner broken ...,28DY
...,...,...
3132,Weir gully needs replacing as it keeps becomin...,2MON
3133,X1 blocked gully nearside verge (DR: Blockage),PAD1
3134,X1 broken gully offside in C/res. Requires mak...,24HC
3135,"X5 gullies require emptying and reseating, lay...",28DY


### Create a Classification column to initally classify flood which have already been alocated a type of failure 

In [91]:
failure_data["Classification"] = ''

In [92]:
distinct_identifiers = []
for row in failure_data["Description"]:
    a = row.find("(")
    b = row.find(")")
    if a >= 0 and b >= 0:
        identifier = row[a:b+1]
        if identifier.find("DR:") != -1:
            failure_data.loc[failure_data["Description"] == row, "Classification"] = identifier
            if identifier not in distinct_identifiers:
                distinct_identifiers.append(identifier)
        else:
            failure_data.loc[failure_data["Description"] == row, "Classification"] = ""
    else:
        failure_data.loc[failure_data["Description"] == row, "Classification"] = ""

In [93]:
distinct_identifiers

['(DR: Blockage)',
 '(DR:Impeded water flow-detritus)',
 '(DR: Obstruction In Drainage)',
 '(DR: Grating/Cover Missing)',
 '(DR: Drainage Collapsed)',
 '(DR: Chamber/benching/pot defec)',
 '(DR: Free flow impeded)',
 '(DR: Loose/Rocking Cover)',
 '(DR: Silted)',
 '(DR: Kerb/channel block align)',
 '(DR: Flooding)',
 '(DR: Inadequately drained)',
 '(DR: Growth likely obstr water)',
 '(DR:Grating/Cover Missing)',
 '(DR: Deformation In Pipe)',
 '(DR: Manhole Shaft defective)']

In [94]:
failure_data["Classification"].value_counts()

                                    1646
(DR: Blockage)                       791
(DR: Drainage Collapsed)             157
(DR: Grating/Cover Missing)          115
(DR: Loose/Rocking Cover)             94
(DR: Free flow impeded)               81
(DR:Impeded water flow-detritus)      78
(DR: Obstruction In Drainage)         44
(DR: Inadequately drained)            39
(DR: Flooding)                        29
(DR: Silted)                          25
(DR: Chamber/benching/pot defec)      24
(DR: Growth likely obstr water)        6
(DR: Kerb/channel block align)         5
(DR: Manhole Shaft defective)          1
(DR:Grating/Cover Missing)             1
(DR: Deformation In Pipe)              1
Name: Classification, dtype: int64

### Simply classifying the (DR: Blockage) failures into full and covered failure modes using key word identification

In [95]:
key_words_covered = ["covered","leaves","Covered","Leaves","Clear","clear","CLEAR","cleared","CLEARED","clearing","tops","top","mouth","mouths",
                     "debris","Debris","silt","Silt","silted","swept","Sweep","SWEPT","SWEEP","sweep","uncover","Uncover","grate","Obstruction","obstructed"]
key_words_full = ["empty","Empty","emptying","Emptying","cleaning out","jet","Jet","jetted","Jetted","Jetting","sucking","Sucking","sucker","SUCKER",
                 "sucked","Sucked","Suck","SUCK","suck","vac","full","FULL","Full"]
key_words_broken = ["collapsing","sunken","Collapsing","Sunken","Reset","reset","re-setting","Broken","broken","replace",
                    "Replace","collapsed","Collapsed","COLLAPSED","pothole","Pothole","POTHOLE","RESET","fit","Fit","FIT",
                   "Adjust","ADJUST","adjust","repair","Repair","missing","Missing","replacing","Replace","REPLACE","REPLACING",
                   "cracked","Cracked","CRACKED","Sandbag","sandbag","Sandbags","SANDBAG","breaking","Breaking","dislodged","Dislodged","misaligned",
                   "Misaligned","degraded","Degraded","frame","Frame","Rattling","rattling","REPAIR","setting","Setting","re-set","Raise",
                    "Hole","hole","patch","Patch","damage","Damage","rocking","Rocking","sunk","Sunk","SUNK","CCTV","cctv","reseating","collapse",
                   "Reinstate","level with","Level with"]
failure_data["covered_key_word_count"] = 0
failure_data["full_key_word_count"] = 0
failure_data["broken_key_word_count"] = 0 
for item in failure_data[failure_data["Classification"] == '(DR: Blockage)']["Description"]:
    count_of_covered_key_words=0
    for key_word in key_words_covered:
        if item.find(key_word) != -1:
            count_of_covered_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "covered_key_word_count"] = count_of_covered_key_words

    count_of_full_key_words=0
    for key_word in key_words_full:
        if item.find(key_word) != -1:
            count_of_full_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "full_key_word_count"] = count_of_full_key_words
    
    count_of_broken_key_words=0
    for key_word in key_words_broken:
        if item.find(key_word) != -1:
            count_of_broken_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "broken_key_word_count"] = count_of_broken_key_words

<b> Rule when both full and covered key words are present <b>
    <br>
    - A gully can be both covered and full meaning in some cases both key words are raised <br>
    - In these cases we will take both failure modes as it is possible for a gully to fail in both ways at once.

In [96]:
print(len(failure_data[(failure_data["full_key_word_count"] == 0) & (failure_data["covered_key_word_count"] == 0)& (failure_data["broken_key_word_count"] == 0) &(failure_data["Classification"] == '(DR: Blockage)')]))
for item in failure_data[(failure_data["full_key_word_count"] == 0) & (failure_data["covered_key_word_count"] == 0)& (failure_data["broken_key_word_count"] == 0) &(failure_data["Classification"] == '(DR: Blockage)')]["Description"]:
    print(item)

217
1 blocked gully (DR: Blockage)
1 x blocked gully. Defect score pre 9 post 9 (DR: Blockage)
2 blocked gully in bus lay-by (DR: Blockage)
1x gully blocked nearside lane one on Toyota exit slip (DR: Blockage)
1x blocked gully lane one nearside verge (DR: Blockage)
2 blocked gully (DR: Blockage)
1x gully blocked on hard shoulder (DR: Blockage)
1 x gully blocked (DR: Blockage)
1x blocked gully in nearside lane one (DR: Blockage)
1x gully blocked nearside slip road (DR: Blockage)
2x gullys blocked (DR: Blockage)
1x blocked gully (DR: Blockage)
1x gully nearside lane one blocked (DR: Blockage)
2x gullies X2 blocked (DR: Blockage)
3 x blocked gully EB 1 x WB (DR: Blockage)
1 end of v channel gully blocked (DR: Blockage)
03/03/2021: Defect: 2no. Weir gullies dug out by highway inspector. (DR: Blockage)
2 gullies blocks (DR: Blockage)
07/07/21 (DR: Blockage)
3 x blocked gullies causing standing water in hard shoulder request to unblock gullies (DR: Blockage)
1x gully in nearside lane one (DR

In [97]:
for item in failure_data[(failure_data["full_key_word_count"] > 1)&(failure_data["covered_key_word_count"] > 1)]["Description"]:
    print(item)

A14 j62 to j61 WB felixstowe l1 lhwt mp 2107 600mm gully is full to the top with silt and not fully clearing water , This will need to be cleared out fully and jetted back to the main carrier,  TM will need to be lane 1 from the roundabout of J62 dock gate 1 ,site is a medium risk (DR: Blockage)
2 gullies Blocked  to the top with silt and weeds  Will need to be dug out and cleaned and the drainage run jetted out.  Tm will need to be a full closure of the slip (DR: Blockage)
Blocked gully full to the top with silt and weeds  Will need to be dug out and cleaned and the drainage run jetted out. can be done within slip road closure in place for patching business case.  (DR: Blockage)
A12 nb Stratford oxbow layby right verge gully 50%full of silt will need to have the catch pit cleared of all silt and the runs jetted to Back to the main carrier to inshore waters can get away ,TM will be needed in the way of a full closure of the layby (DR: Blockage)
Blocked gullies and carrier pipe within t

In [98]:
for item in key_words_broken:
    if item in "Blocked gullies and carrier pipe within the lead in slip to the oxbow lay-by . We will need to remove the gully grates and fully clear the gully pots jet and clear the drainage pipes back to the carrier drain all catch pits to be cleared and drainage runs jetted though out the whole lay-by but special attention should  be placed on the area of flooding.  The lay-by is currently closed and as such day time working will be appropriate (DR: Blockage)":
        print(item)
    

hole


In [99]:
failure_data[(failure_data["full_key_word_count"] == 0) & (failure_data["covered_key_word_count"] == 0)&(failure_data["Classification"] == '(DR: Blockage)')]

,Description,Priority,Classification,covered_key_word_count,full_key_word_count,broken_key_word_count
27,1 blocked gully (DR: Blockage),24HR,(DR: Blockage),0,0,0
57,1 x blocked gully. Defect score pre 9 post 9 (...,PAD1,(DR: Blockage),0,0,0
61,2 blocked gully in bus lay-by (DR: Blockage),28DY,(DR: Blockage),0,0,0
63,1x gully blocked nearside lane one on Toyota e...,PAD1,(DR: Blockage),0,0,0
65,1x blocked gully lane one nearside verge (DR: ...,PAD1,(DR: Blockage),0,0,0
...,...,...,...,...,...,...
3105,Sunblock gully in 28 days (DR: Blockage),28CO,(DR: Blockage),0,0,0
3113,Various blocked gullies verge side including o...,PAD1,(DR: Blockage),0,0,0
3116,Various blocked gullies and kerb drains eb and...,PAD1,(DR: Blockage),0,0,0
3129,Unblock arterial gully (DR: Blockage),FNOW,(DR: Blockage),0,0,0


There are 219 rows left.

In [100]:
drain_broken = ["(DR: Kerb/channel block align)","(DR:Grating/Cover Missing)","(DR: Deformation In Pipe)","(DR: Manhole Shaft defective)",
               "(DR: Chamber/benching/pot defec)","(DR: Loose/Rocking Cover)","(DR: Grating/Cover Missing)",'(DR: Drainage Collapsed)']
drain_covered = ["(DR: Growth likely obstr water)","(DR: Silted)",'(DR: Inadequately drained)',"(DR: Obstruction In Drainage)",
                "(DR:Impeded water flow-detritus)","(DR: Free flow impeded)"]

In [101]:
key_words_covered = ["covered","leaves","Covered","Leaves","Clear","clear","cleared","clearing","CLEARED","CLEARING","tops","top","mouth","mouths","debris","Debris","vegetation","Vegetation"]
key_words_full = ["empty","Empty","emptying","Emptying","cleaning out","jet","Jet","jetted","Jetted","Jetting","JET","SUCK","sucking","Sucking","sucker","Sucker","sucked","Sucked"]
failure_data["covered_key_word_count"] = 0
failure_data["full_key_word_count"] = 0
for item in failure_data[failure_data["Classification"] == ""]["Description"]:
    count_of_covered_key_words=0
    for key_word in key_words_covered:
        if item.find(key_word) != -1:
            count_of_covered_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "covered_key_word_count"] = count_of_covered_key_words

    count_of_full_key_words=0
    for key_word in key_words_full:
        if item.find(key_word) != -1:
            count_of_full_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "full_key_word_count"] = count_of_full_key_words

In [102]:
failure_data[(failure_data["Classification"] == "")&((failure_data["covered_key_word_count"] > 0 ) | (failure_data["full_key_word_count"]>0))]

,Description,Priority,Classification,covered_key_word_count,full_key_word_count,broken_key_word_count
2,(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage...,28DY,,1,0,0
7,1 x gully blocked/partially blocked by vegetat...,28DY,,1,1,0
8,(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage...,28DY,,1,0,0
14,(Gully) LOG 27 - 1 No. New gully lid required ...,28DY,,0,1,0
17,(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage...,28DY,,1,0,0
...,...,...,...,...,...,...
3127,Two x gully’s made safe by removing silt and d...,FNOW,,1,0,0
3128,Silted gully top (Drainage Defect),FNOW,,1,0,0
3130,Vehicle fire resulting in superficial road sur...,28DY,,0,1,0
3135,"X5 gullies require emptying and reseating, lay...",28DY,,0,2,0


# Investigate using drainage defect as a word classifier

In [103]:
key_words_covered = ["covered","leaves","Covered","Leaves","Clear","clear","CLEAR","cleared","CLEARED","clearing","cleaning","Cleaning","tops","top","mouth","mouths",
                     "debris","Debris","silt","Silt","silted","swept","Sweep","SWEPT","SWEEP","sweep","uncover","Uncover","grate","Obstruction","obstructed","detritus","Detritus","DETRITUS","Free flow impeded"]
key_words_full = ["empty","Empty","emptying","Emptying","gully cleaning","cleaning out","jet","Jet","jetted","Jetted","Jetting","sucking","Sucking","sucker","SUCKER",
                 "sucked","Sucked","Suck","SUCK","suck","vac"] #-- silt probably is in full
key_words_broken = ["collapsing","sunken","Collapsing","Sunken","Reset","reset","re-setting","Broken","broken","replace",
                    "Replace","collapsed","Collapsed","COLLAPSED","pothole","Pothole","POTHOLE","RESET","fit","Fit","FIT",
                   "Adjust","ADJUST","adjust","repair","Repair","missing","Missing","replacing","Replace","REPLACE","REPLACING",
                   "cracked","Cracked","CRACKED","Sandbag","sandbag","Sandbags","SANDBAG","breaking","Breaking","dislodged","Dislodged","misaligned",
                   "Misaligned","degraded","Degraded","frame","Frame","Rattling","rattling","REPAIR","setting","Setting","re-set","Raise",
                    "Hole","hole","patch","Patch","damage","Damage","rocking","Rocking","sunk","Sunk","SUNK","CCTV","cctv","reseating","collapse",
                   "Reinstate","level with","Level with","unseize","seize","Unseize","Seize","broken","Drainage Defect"]


key_words_blocked = ["blocked","Blocked","blockage","Blockage","unblock","Unblock","blocking","Blockage"]

failure_data["blocked_key_word_count"] = 0
failure_data["covered_key_word_count"] = 0
failure_data["full_key_word_count"] = 0
failure_data["broken_key_word_count"] = 0 
for item in failure_data[failure_data["Classification"] == '']["Description"]:
    count_of_covered_key_words=0
    for key_word in key_words_covered:
        if item.find(key_word) != -1:
            count_of_covered_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "covered_key_word_count"] = count_of_covered_key_words

    count_of_full_key_words=0
    for key_word in key_words_full:
        if item.find(key_word) != -1:
            count_of_full_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "full_key_word_count"] = count_of_full_key_words
    
    count_of_broken_key_words=0
    for key_word in key_words_broken:
        if item.find(key_word) != -1:
            count_of_broken_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "broken_key_word_count"] = count_of_broken_key_words
    
    count_of_blocked_key_words=0
    for key_word in key_words_blocked:
        if item.find(key_word) != -1:
            count_of_blocked_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "blocked_key_word_count"] = count_of_blocked_key_words


# Update classification after converstaion with Mike 

In [110]:
failure_data = pd.read_csv("../project_data/drainage_failures.csv",names=["Description","Priority"])

In [111]:
key_words_covered = ["covered","leaves","Covered","Leaves","Clear","clear","CLEAR","cleared","CLEARED","clearing","cleaning","Cleaning","tops","top","mouth","mouths",
                     "debris","Debris","swept","Sweep","SWEPT","SWEEP","sweep","uncover","Uncover","grate","Obstruction","obstructed","detritus","Detritus","DETRITUS","Free flow impeded"]
key_words_full = ["empty","Empty","emptying","Emptying","gully cleaning","cleansing","cleaning out","jet","Jet","jetted","Jetted","Jetting","sucking","Sucking","sucker","SUCKER",
                 "sucked","Sucked","Suck","SUCK","suck","vac","blocked","Blocked","blockage","Blockage","unblock","Unblock","blocking","Blockage","Silt","silted","silt","Clean out gully"]
key_words_broken = ["collapsing","sunken","Collapsing","Sunken","Reset","reset","re-setting","Broken","broken","replace",
                    "Replace","collapsed","Collapsed","COLLAPSED","pothole","Pothole","POTHOLE","RESET","fit","Fit","FIT",
                   "Adjust","ADJUST","adjust","repair","Repair","missing","Missing","replacing","Replace","REPLACE","REPLACING",
                   "cracked","Cracked","CRACKED","Sandbag","sandbag","Sandbags","SANDBAG","breaking","Breaking","dislodged","Dislodged","misaligned",
                   "Misaligned","degraded","Degraded","frame","Frame","Rattling","rattling","REPAIR","setting","Setting","re-set","Raise",
                    "Hole","hole","patch","Patch","damage","Damage","rocking","Rocking","sunk","Sunk","SUNK","CCTV","cctv","reseating","collapse",
                   "Reinstate","level with","Level with","unseize","seize","Unseize","Seize","broken","New gully lid required","no lid","No lid","lid open","Lid popped out"]

failure_data["covered_key_word_count"] = 0
failure_data["full_key_word_count"] = 0
failure_data["broken_key_word_count"] = 0 
for item in failure_data["Description"]:
    count_of_covered_key_words=0
    for key_word in key_words_covered:
        if item.find(key_word) != -1:
            count_of_covered_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "covered_key_word_count"] = count_of_covered_key_words

    count_of_full_key_words=0
    for key_word in key_words_full:
        if item.find(key_word) != -1:
            count_of_full_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "full_key_word_count"] = count_of_full_key_words
    
    count_of_broken_key_words=0
    for key_word in key_words_broken:
        if item.find(key_word) != -1:
            count_of_broken_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "broken_key_word_count"] = count_of_broken_key_words

In [113]:
key_words_silted = ["Silt","silted","silt","SILT"]
for item in failure_data[(failure_data["covered_key_word_count"]==1)&(failure_data["full_key_word_count"]==1)&(failure_data["broken_key_word_count"]==0)]["Description"]:
    for key_word in key_words_silted:
        if item.find(key_word) != -1:
            print(item)

2 x gully’s covered in silt made safe by removing and letting water flow freely (Drainage Defect)
1x gully lane one nearside on verge built up of silt on top of gully lid (Drainage Defect)
1x gully nearside lane one on vege built up of silt on top of gully lid (Drainage Defect)
Crew are to remove any debris / silt / weeds, that are impeding the flow of water into any point drainage assets on the hard shoulder. Ensuring that debris is placed at least a metre away from the asset.
Crossover at BP garage Aco drainage requires cleaning over half the 20 Mts section silt with Aco drain - this is a Flood Spot  Area , within  works 1 x gully chamber also needs cleaning as Aco drain deposits silt into this chamber . (Drainage Defect)
Crew are to strim brambles and vegetation to gain access to the terminal gully. Crew are then to scrape the silt off the terminal gully and dispose of the debris / arisings.
Crew are to remove any debris / silt / weeds, that are impeding the flow of water into any p

In [108]:
for item in failure_data[(failure_data["covered_key_word_count"]==0)&(failure_data["full_key_word_count"]==0)&(failure_data["broken_key_word_count"]==0)]["Description"]:
    print(item)

 (Enquiry 12121430)Please provide flood boards in advance of the layby,  Another job will be raised to carry out further investigation 
24/12/2020: Defect: flooding to 100% of lane 1for approximately 40lm. (DR: Flooding)
A1 NB 96/8 (Drainage Defect)
2 x gullys on chevrons requires making safe. R.o.c log 568. Pre score 12 Post score 12 (Drainage Defect)
Aquakerb lid off (Drainage Defect)
Flooding across lane 1 Put out flood warning signs in 24hours MATRIX 18 (Drainage Defect)
Between MP190/0 - 190/5 please remove all growth from V channel and gullies to allow free flow of water, thats building up.(Drainage Defect)
clean gully and return to normal opperations remove arrisings from site
Drain (Drainage Defect)
Flooding into live lane that requires a make safe by the service provider . (Drainage Defect)
DR: Inadequately drained
Can all gully’s LE Lane1 be cleaned and ensured free flow. (DR: Inadequately drained)
Flooding across slip road (Drainage Defect)
Flooding in hard shoulder and lane

In [109]:
failure_data

,Description,Priority,Classification,covered_key_word_count,full_key_word_count,broken_key_word_count,blocked_key_word_count
0,1 x 600 x 600 sunken and collapsing gully requ...,28DY,,0,0,3,0
1,***This needs to be a E600*** Gully grating da...,28DY,,0,0,3,0
2,(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage...,28DY,,1,3,0,1
3,(Gully) 1 No new lid required for seized gully...,28DY,,0,4,1,4
4,(Gully) 1 No. new lid required. Corner broken ...,28DY,,0,0,3,0
...,...,...,...,...,...,...,...
3132,Weir gully needs replacing as it keeps becomin...,2MON,(DR: Obstruction In Drainage),1,0,2,0
3133,X1 blocked gully nearside verge (DR: Blockage),PAD1,(DR: Blockage),0,3,0,0
3134,X1 broken gully offside in C/res. Requires mak...,24HC,(DR: Drainage Collapsed),1,0,3,0
3135,"X5 gullies require emptying and reseating, lay...",28DY,,0,2,1,0


In [70]:
failure_data.head(50)

,Description,Priority,Classification,covered_key_word_count,full_key_word_count,broken_key_word_count
0,1 x 600 x 600 sunken and collapsing gully requ...,28DY,,0,0,3
1,***This needs to be a E600*** Gully grating da...,28DY,,0,0,3
2,(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage...,28DY,,0,2,0
3,(Gully) 1 No new lid required for seized gully...,28DY,,0,4,1
4,(Gully) 1 No. new lid required. Corner broken ...,28DY,,0,0,3
5,1 No. broken side entry gully requires repair....,28DY,,0,0,3
6,(Enquiry 12121430) Please investigate blockag...,7DY,,0,1,1
7,1 x gully blocked/partially blocked by vegetat...,28DY,,1,3,0
8,(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage...,28DY,,0,2,0
9,2 x Blocked gullies 100% full. Requires empty...,24HR,(DR: Blockage),0,5,0


In [25]:
for item in failure_data[failure_data["covered_key_word_count"]>1]["Description"]:
    if item.find("silt") != -1:
        print(item)
    elif item.find("Silt") != -1:
        print(item)
    elif item.find("SILT") != -1:
        print(item)
    else:
        pass

(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage Road, Nearside Gully full of Silt/Detritus & Vegetation, appears Blocked.  Approximately 45m after end of Splitter.  Defect Score: 12 (Consequence: Medium, Likelihood: Medium) 28 Day Gully/Drainage System requires cleansing/maintenance. (Drainage Defect)
1 x gully blocked/partially blocked by vegetation, silt and detritus. Gully cleansing required & jetting (Drainage Defect)
(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage Road, Nearside Gully full of Silt/Detritus & Vegetation, appears Blocked.  Opposite Sign UL1009..  Defect Score: 12 (Consequence: Medium, Likelihood: Medium) 28 Day Gully/Drainage System requires cleansing/maintenance. (Drainage Defect)
(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage Road, Nearside Gully full of Silt/Detritus & Vegetation, appears Blocked.  Approximately 15m after end of Splitter.  Defect Score: 12 (Consequence: Medium, Likelihood: Medium) 28 Day Gully/Drainage System requires cleansing/maintenance. (Draina

In [28]:
for item in failure_data[failure_data["blocked_key_word_count"] > 0]["Description"]:
    print(item)

(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage Road, Nearside Gully full of Silt/Detritus & Vegetation, appears Blocked.  Approximately 45m after end of Splitter.  Defect Score: 12 (Consequence: Medium, Likelihood: Medium) 28 Day Gully/Drainage System requires cleansing/maintenance. (Drainage Defect)
(Gully) 1 No new lid required for seized gully. Also requires unblocking (DR: Blockage). Marked with an 'x' for I.D.  - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
 (Enquiry 12121430) Please investigate blockage to drainage pipe and repair if possible, remove flood boards if work has been succesful. Please confirm the issue and provide photos of the work. TM A lane closure will be required.
1 x gully blocked/partially blocked by vegetation, silt and detritus. Gully cleansing required & jetting (Drainage Defect)
(Enquiry 19008696) A5 EB M6 Jct 12 to Vicarage Road, Nearside Gully full of Silt/Detritus & Vegetation, appears Blocked.  Oppos

In [53]:
print(len(failure_data[(failure_data["Classification"]=='')&(failure_data["covered_key_word_count"] == 0) & (failure_data["full_key_word_count"]==0) & (failure_data["broken_key_word_count"]==0)]))
for item in failure_data[(failure_data["Classification"]=='')&(failure_data["covered_key_word_count"] == 0) & (failure_data["full_key_word_count"]==0) & (failure_data["broken_key_word_count"]==0)]["Description"]:
    print(f"desc: {item}")

40
desc: (Gully) 1 No. gully requires unblocking (DR: Blockage) - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
desc:  (Enquiry 12121430)Please provide flood boards in advance of the layby,  Another job will be raised to carry out further investigation 
desc: (Gully) 1 No. New gully lid required - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
desc: (Gully) LOG 7 - 1 No. New gully lid required. Marked for I.D. with yellow paint - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation.   
desc: (Gully) 1 No. New gully lid required. INCLUDES TM FOR JOB 41402153 - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
desc: (Gully) 1 No. New gully lid required. Gully is third from start of layby - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
desc: Block

In [54]:
print(len(failure_data[(failure_data["Classification"]=='')&(failure_data["covered_key_word_count"] == 0) & (failure_data["full_key_word_count"]==0) & (failure_data["broken_key_word_count"]==0) & (failure_data["blocked_key_word_count"]==0)]))
for item in failure_data[(failure_data["Classification"]=='')&(failure_data["covered_key_word_count"] == 0) & (failure_data["full_key_word_count"]==0) & (failure_data["broken_key_word_count"]==0) & (failure_data["blocked_key_word_count"]==0)]["Description"]:
    print(f"desc: {item}")

23
desc:  (Enquiry 12121430)Please provide flood boards in advance of the layby,  Another job will be raised to carry out further investigation 
desc: (Gully) 1 No. New gully lid required - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
desc: (Gully) LOG 7 - 1 No. New gully lid required. Marked for I.D. with yellow paint - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation.   
desc: (Gully) 1 No. New gully lid required. INCLUDES TM FOR JOB 41402153 - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
desc: (Gully) 1 No. New gully lid required. Gully is third from start of layby - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
desc: DR: Drainage Systems - MAKE SAFE - offside gully with no lid encroaching live line.  (Enquiry 14592216)
desc: clean gully and return to normal opperations remove arri

# Analyse current word search claissifcation method to whole dataset without classifiers

In [57]:
key_words_covered = ["covered","leaves","Covered","Leaves","Clear","clear","CLEAR","cleared","CLEARED","clearing","cleaning","Cleaning","tops","top","mouth","mouths",
                     "debris","Debris","silt","Silt","silted","swept","Sweep","SWEPT","SWEEP","sweep","uncover","Uncover","grate","Obstruction","obstructed","detritus","Detritus","DETRITUS","Free flow impeded"]
key_words_full = ["empty","Empty","emptying","Emptying","gully cleaning","cleaning out","jet","Jet","jetted","Jetted","Jetting","sucking","Sucking","sucker","SUCKER",
                 "sucked","Sucked","Suck","SUCK","suck","vac"]
key_words_broken = ["collapsing","sunken","Collapsing","Sunken","Reset","reset","re-setting","Broken","broken","replace",
                    "Replace","collapsed","Collapsed","COLLAPSED","pothole","Pothole","POTHOLE","RESET","fit","Fit","FIT",
                   "Adjust","ADJUST","adjust","repair","Repair","missing","Missing","replacing","Replace","REPLACE","REPLACING",
                   "cracked","Cracked","CRACKED","Sandbag","sandbag","Sandbags","SANDBAG","breaking","Breaking","dislodged","Dislodged","misaligned",
                   "Misaligned","degraded","Degraded","frame","Frame","Rattling","rattling","REPAIR","setting","Setting","re-set","Raise",
                    "Hole","hole","patch","Patch","damage","Damage","rocking","Rocking","sunk","Sunk","SUNK","CCTV","cctv","reseating","collapse",
                   "Reinstate","level with","Level with","unseize","seize","Unseize","Seize","broken","Drainage Defect"]
key_words_blocked = ["blocked","Blocked","blockage","Blockage","unblock","UNBLOCKING","BLOCKED","UNBLOCK","Unblock","blocking","Blockage"]

failure_data["blocked_key_word_count"] = 0
failure_data["covered_key_word_count"] = 0
failure_data["full_key_word_count"] = 0
failure_data["broken_key_word_count"] = 0 
for item in failure_data["Description"]:
    count_of_covered_key_words=0
    for key_word in key_words_covered:
        if item.find(key_word) != -1:
            count_of_covered_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "covered_key_word_count"] = count_of_covered_key_words

    count_of_full_key_words=0
    for key_word in key_words_full:
        if item.find(key_word) != -1:
            count_of_full_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "full_key_word_count"] = count_of_full_key_words
    
    count_of_broken_key_words=0
    for key_word in key_words_broken:
        if item.find(key_word) != -1:
            count_of_broken_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "broken_key_word_count"] = count_of_broken_key_words
    
    count_of_blocked_key_words=0
    for key_word in key_words_blocked:
        if item.find(key_word) != -1:
            count_of_blocked_key_words += 1
    failure_data.loc[failure_data["Description"] == item, "blocked_key_word_count"] = count_of_blocked_key_words


In [56]:
failure_data[(failure_data["covered_key_word_count"]==0)&(failure_data["full_key_word_count"]==0)&(failure_data["broken_key_word_count"]==0)]

,Description,Priority,Classification,covered_key_word_count,full_key_word_count,broken_key_word_count,blocked_key_word_count
24,(Gully) 1 No. gully requires unblocking (DR: B...,28DY,,0,0,0,4
27,1 blocked gully (DR: Blockage),24HR,(DR: Blockage),0,0,0,3
52,(Enquiry 12121430)Please provide flood boards...,24HR,,0,0,0,0
57,1 x blocked gully. Defect score pre 9 post 9 (...,PAD1,(DR: Blockage),0,0,0,3
61,2 blocked gully in bus lay-by (DR: Blockage),28DY,(DR: Blockage),0,0,0,3
...,...,...,...,...,...,...,...
3105,Sunblock gully in 28 days (DR: Blockage),28CO,(DR: Blockage),0,0,0,3
3113,Various blocked gullies verge side including o...,PAD1,(DR: Blockage),0,0,0,3
3116,Various blocked gullies and kerb drains eb and...,PAD1,(DR: Blockage),0,0,0,3
3129,Unblock arterial gully (DR: Blockage),FNOW,(DR: Blockage),0,0,0,3


In [157]:
classically_blocked = ["blocked","unblocking","Blocked","BLOCKED","UNBLOCKING","Unblocking","Blockage","blockage","block","Block"]

for item in failure_data[(failure_data["covered_key_word_count"]==0)&(failure_data["full_key_word_count"]==0)&(failure_data["broken_key_word_count"]==0)]["Description"]:
    word_count = 0
    for word in classically_blocked:
        if word in item:
            word_count += 1
        else:
            pass
    if word_count == 0:
        print(item)

 (Enquiry 12121430)Please provide flood boards in advance of the layby,  Another job will be raised to carry out further investigation 
1 x Gully lid seized after attempted to clean during cleaning (DR: Chamber/benching/pot defec)
(Gully) 1 No. New gully lid required - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
(Gully) LOG 7 - 1 No. New gully lid required. Marked for I.D. with yellow paint - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation.   
(Gully) 1 No. New gully lid required. INCLUDES TM FOR JOB 41402153 - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
2 x 3 wayTraffic light closure to complete gully cleaning
(Gully) 1 No. New gully lid required. Gully is third from start of layby - All works are to comply with CDM 2015 requirements and all relevant health & safety legislation. 
24/12/2020: Defect: flooding to 100% of lane 1for ap

# Using the SOR codes to understand whether there is a correlation between failure mode and SOR code

In [204]:
enhanced_failure_data = pd.read_csv("../project_data/drainage_failures_enhanced.csv",names=["Region","Description","Priority","SOR"])

In [205]:
merged = pd.merge(failure_data,enhanced_failure_data, on="Description")

In [206]:
merged

,Description,Priority_x,Classification,covered_key_word_count,full_key_word_count,broken_key_word_count,Region,Priority_y,SOR
0,1 x 600 x 600 sunken and collapsing gully requ...,28DY,,0,0,3,MD09,28DY,C03.013
1,***This needs to be a E600*** Gully grating da...,28DY,,0,0,2,SO03,28DY,C03.009
2,***This needs to be a E600*** Gully grating da...,28DY,,0,0,2,SO03,28DY,C04.001
3,***This needs to be a E600*** Gully grating da...,28DY,,0,0,2,SO03,28DY,CTM04507
4,***This needs to be a E600*** Gully grating da...,28DY,,0,0,2,SO03,28DY,CTM04512
...,...,...,...,...,...,...,...,...,...
10595,"X2 broken gully tops, unable to clarify measur...",28DY,,2,0,1,SO03,28DY,CTM04383
10596,"X2 broken gully tops, unable to clarify measur...",28DY,,2,0,1,SO03,28DY,CTM04478
10597,"X2 broken gully tops, unable to clarify measur...",28DY,,2,0,1,SO03,28DY,CTM04484
10598,"X2 broken gully tops, unable to clarify measur...",28DY,,2,0,1,SO03,28DY,CTM04497


In [207]:
for region in merged["Region"].unique():
    print(region, merged[(merged["broken_key_word_count"]>0)&(merged["Region"]==region)]["SOR"].unique())
    break

MD09 ['C03.013' 'C10.003' 'C10.007' 'C03.010' 'C03.016' 'C03.067' 'DCR.01'
 'DCR.02' 'DCR.03' 'DCR.04' 'DCR.05' 'C03.012' 'C04.001' 'C03.014'
 'FIXNOW' nan 'C03.070A']


In [208]:
for region in merged["Region"].unique():
    print(region, merged[(merged["full_key_word_count"]>0)&(merged["Region"]==region)]["SOR"].unique())
    break

MD09 ['C03.016' 'C03.067' 'C03.010' 'C03.070A' 'C03.070' 'C03.070D' 'C03.141'
 'C03.145' 'DCR.01' 'DCR.02' 'DCR.03' 'DCR.04' 'DCR.05' 'C03.069'
 'C03.142' 'C03.146' 'C03.017']


In [209]:
for region in merged["Region"].unique():
    print(region, merged[(merged["covered_key_word_count"]>0)&(merged["Region"]==region)]["SOR"].unique())
    break

MD09 ['C03.067' 'C03.016' 'C03.070A' 'DCR.01' 'DCR.02' 'DCR.03' 'DCR.04'
 'DCR.05' 'FIXNOW' 'C03.069' 'C03.141' 'C03.145' 'C03.070' 'C03.070D'
 'C10.003' 'C10.007' 'NCA' 'C03.017' 'C04.001']


<b>As you can see, basically the same sor items are present for each of the failure modes

In [210]:
for region in merged["Region"].unique():
    print(region, merged[(merged["broken_key_word_count"]>0)&(merged["full_key_word_count"]==0)&(merged["covered_key_word_count"]==0)&(merged["Region"]==region)]["SOR"].unique())
    break

MD09 ['C03.013' 'C10.003' 'C10.007' 'C03.012' 'C04.001' 'DCR.01' 'DCR.02'
 'DCR.03' 'DCR.04' 'DCR.05' 'C03.014' 'C03.010' 'FIXNOW' nan 'C03.067'
 'C03.070A']


In [211]:
for region in merged["Region"].unique():
    print(region, merged[(merged["broken_key_word_count"]==0)&(merged["full_key_word_count"]>0)&(merged["covered_key_word_count"]==0)&(merged["Region"]==region)]["SOR"].unique())
    break

MD09 ['C03.067' 'C03.070A' 'C03.141' 'C03.145' 'DCR.01' 'DCR.02' 'DCR.03'
 'DCR.04' 'DCR.05' 'C03.016' 'C03.069' 'C03.142' 'C03.146']


In [212]:
for region in merged["Region"].unique():
    print(region, merged[(merged["broken_key_word_count"]==0)&(merged["full_key_word_count"]==0)&(merged["covered_key_word_count"]>0)&(merged["Region"]==region)]["SOR"].unique())
    break

MD09 ['C03.067' 'C03.070A' 'C03.016' 'FIXNOW' 'C03.069' 'C03.141' 'C03.145'
 'DCR.01' 'DCR.02' 'DCR.03' 'DCR.04' 'DCR.05' 'C03.070' 'C10.003'
 'C10.007' 'C03.070D' 'NCA' 'C04.001']


<b>After constraining it to one and only one failure mode per sor there is some constraining of the sor codes however there is not enough to change classification methods. 